In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import datetime
%load_ext tensorboard

## Keras Models and Layers
- In this Notebook we will examine how keras uses tf.Module.
- tf.keras.layers.Layer is the base class of all keras layers and it inherits from tf.Moudle
- We can convert a module into a Keras layer just by swapping out the parent and then changing __call__ to call:

In [5]:
class MyDense(keras.layers.Layer):
    # adding **kwargs to support base keras layer arguments
    def __init__(self, in_features, out_features, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        # This will soon move to build step:
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')

    def call(self, x):
        y = tf.matmul(x, self.w)+self.b
        return tf.nn.relu(y)
simple_layer = MyDense(name="Simple", in_features=3, out_features=3)
# Keras layers have their own __call__ that does some bookkeeping described in the next section and then calls call(). You should notice no change in functionality.

In [6]:
simple_layer([[2.0,2.0,2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.       , 0.       , 1.0608517]], dtype=float32)>

### The Build Step
- As noted it's convenient in many cases to wait to create variables until you are sure of the input shape
- Keras layers come with extra lifecycle step that allows you more flexibility in how you define your layers.
- This is defined in the build function
- build is called exactly once, and it is called with the shape of the input. it's usually used to create variables (weights)
- We can rewrite MyDense layer above to be flexible to the size of its inputs:

In [7]:
class FlexibleDense(keras.layers.Layer):
    # Note the added `**kwargs`, as Keras supports many arguments
    def __init__(self, out_features, **kwargs):
        super(FlexibleDense, self).__init__(**kwargs)
        self.out_features = out_features
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
# Create an instance of the layer
flexible_dense = FlexibleDense(out_features=3)
#At this point, the model has not been built, so there are no variables:
flexible_dense.variables

[]

In [8]:
# Calling the function allocates appropriately-sized variables:
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[-0.31626916  3.6292317   0.11567521]
 [-0.4744041   5.443848    0.17351276]], shape=(2, 3), dtype=float32)


In [9]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-1.5037673 ,  1.0501281 , -0.6804019 ],
        [ 0.2773653 ,  0.8656277 ,  0.5130308 ],
        [ 1.0682673 , -0.10113989,  0.2252087 ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [10]:
# Since build is only called once, inputs will be rejected if the input shape is not compatible with the layer's variables:
try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

Failed: Exception encountered when calling layer "flexible_dense" (type FlexibleDense).

Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]

Call arguments received by layer "flexible_dense" (type FlexibleDense):
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)


Keras layers have a lot more extra features including:  
    - Optional losses  
    - Support for metrics  
    - Built-in support for an optional training argument to differentiate between training and inference use  
    - get_config and from_config methods that allow you to accurately store configurations to allow model cloning in Python  

### Keras Models
- We can define your model as nested Keras layers.
- However, Keras also provides a full-featured model class called tf.keras.Model. 
- It inherits from tf.keras.layers.Layer, so a Keras model can be used, nested, and saved in the same way as Keras layers. 
- Keras models come with extra functionality that makes them easy to train, evaluate, load, save, and even train on multiple machines.
- We can define the SequentialModule from above with nearly identical code, again converting __call__ to call() and changing the parent:

In [11]:
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[-10.384374   -1.9303488]], shape=(1, 2), dtype=float32)


In [12]:
# All the same features are available, including tracking variables and submodules.
# Note: To emphasize the note above, a raw tf.Module nested inside a Keras layer or model will not get its variables collected for training or saving. 
# Instead, nest Keras layers inside of Keras layers.
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.6732569 , -0.74500614, -1.0023254 ],
        [-0.72728425,  0.5621365 , -0.8005991 ],
        [ 1.6960504 ,  0.11627781, -0.75572777]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-1.3194591 ,  0.35605294],
        [-1.1395929 ,  0.7418711 ],
        [ 1.2121584 ,  0.5864098 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [13]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x23b0f9f4f10>,
 <__main__.FlexibleDense at 0x23b0f722ac0>)

In [14]:
# Overriding tf.keras.Model is a very Pythonic approach to building TensorFlow models. If you are migrating models from other frameworks, this can be very straightforward.
# If you are constructing models that are simple assemblages of existing layers and inputs, you can save time and space by using the functional API, 
# which comes with additional features around model reconstruction and architecture.
# Here is the same model with the functional API:

inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 flexible_dense_3 (FlexibleD  (None, 3)                12        
 ense)                                                           
                                                                 
 flexible_dense_4 (FlexibleD  (None, 2)                8         
 ense)                                                           
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [15]:
# The major difference here is that the input shape is specified up front as part of the functional construction process. 
# The input_shape argument in this case does not have to be completely specified; you can leave some dimensions as None.
# Note: You do not need to specify input_shape or an InputLayer in a subclassed model; these arguments and layers will be ignored.
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.856375, 9.040522]], dtype=float32)>

### Saving Keras Models
- Keras models can be checkpointed, and that will look the same as tf.Module.
- Keras models can also be saved with tf.saved_model.save(), as they are modules. 
- However, Keras models have convenience methods and other functionality:

In [16]:
my_sequential_model.save("exname_of_file")


INFO:tensorflow:Assets written to: exname_of_file\assets


In [17]:
# Just as easily, they can be loaded back in:
reconstructed_model = tf.keras.models.load_model("exname_of_file")

In [18]:
# Keras SavedModels also save metric, loss, and optimizer states.
# This reconstructed model can be used and will produce the same result when called on the same data:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-10.384374 ,  -1.9303488]], dtype=float32)>